## Imporing neccessary libraries

In [5]:
import pandas as pd
import numpy as np
import os
import re
import string
import pickle
from nltk.corpus import stopwords
import nltk
from gensim.models.doc2vec import Doc2Vec
nltk.download('stopwords')
nltk.download('punkt')
remove_these = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
nltk.download('omw-1.4')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

## Text Preprocessing

In [2]:
def preprocess(text):
  wordnet_lemmatizer = WordNetLemmatizer()
  text = text.lower()
  tokens = word_tokenize(text)
  tokens = [i for i in tokens if i not in string.punctuation]
  tokens = [w for w in tokens if w not in remove_these]
  preprocess_text = [wordnet_lemmatizer.lemmatize(word) for word in tokens]
  preprocess_text = ' '.join(preprocess_text)
  return preprocess_text

## Loading models

In [13]:
# Loading the models which were saved during training
model = Doc2Vec.load('doc_vec_model')
scaler = pickle.load(open('scaler.pkl', 'rb'))
kmeans_model = pickle.load(open("kmeans_model_500.pkl",'rb'))
pca = pickle.load(open('pca.pkl', 'rb'))

In [3]:
# Add your lyrics to test
lyrics = """You should let me love you
Let me be the one to (Let me be the one to give)
Give you everything you want and need (Everything you need)
Baby, good love and protection (Said everything)
Make me your selection (Will you take me, baby?)
Show you the way love's supposed to be (Oh)
Baby, you should let me love you
Let me be the one to
Give you everything you want and need (Give you everything)
That good love and protection
Make me your selection
Show you the way love's supposed to be (Hey)
You should let me love you (Love you)
Let me be the one to give you everything you want and need
(The one to, everything you need)
Good love and protection (Protection)
Make me your selection (Selection, baby)
Show you the way love's supposed to be, yeah (Oh)
Let me love you
That's all you need, baby"""

## Get embeddings

In [6]:
cleaned_text = preprocess(lyrics)

In [9]:
df = pd.DataFrame({'cleaned_text': cleaned_text},index=[0])

In [14]:
embeddings_df = pd.DataFrame([model.infer_vector(cleaned_text)])

In [15]:
embeddings_df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-1.243637,-0.645911,0.519882,0.238891,-0.157395,0.120959,0.567404,1.07219,-0.848339,0.245386,...,0.639373,0.519487,0.217612,-0.700756,-0.025395,-0.073664,-0.485191,-1.22957,-0.181185,0.110586


## Transforming Embeddings

In [16]:
song_embedding = scaler.transform(embeddings_df)

In [17]:
song_embedding = pca.transform(song_embedding)

In [18]:
song_embedding

array([[ 3.8029668 , -0.26760358, -0.05825629]], dtype=float32)

In [20]:
song_df  = pd.DataFrame(data = song_embedding)


In [21]:
song_df

,0,1,2
0,3.802967,-0.267604,-0.058256


## Recommending Songs

In [22]:
predicted_label = (kmeans_model.predict(song_df)).tolist()

In [23]:
predicted_label

[458]

In [24]:
song_selection = pd.read_csv('songs_selection_500.csv')

In [25]:
song_selection.drop('Unnamed: 0', axis=1,inplace=True)

In [26]:
song_selection = song_selection[['label','title']]

In [27]:
songs_list = song_selection[song_selection['label']==predicted_label[0]]

In [29]:
songs_list

,label,title
562,458,Guiding Light (LP Version)
841,458,Stormy Weather
888,458,The Right Combination
1173,458,P.U.S.A.
1509,458,Love Can Break Your Heart (LP Version)
...,...,...
49778,458,Burn
49785,458,Forever
49807,458,Big Sleep
49855,458,Have A Little Faith


In [28]:
np.random.choice(songs_list['title'],3)

array(['96 tears', 'End It On This', 'Ride Your Pony'], dtype=object)